In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('max_columns', 100)

In [2]:
df = pd.read_csv('data/raw/police_reports.csv')
df.columns = [c.lower().replace(" ","_").strip() for c in df.columns]
print(df.shape)
df.head()

(333016, 36)


,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,cad_number,report_type_code,report_type_description,filed_online,incident_code,incident_category,incident_subcategory,incident_description,resolution,intersection,cnn,police_district,analysis_neighborhood,supervisor_district,latitude,longitude,point,sf_find_neighborhoods,current_police_districts,current_supervisor_districts,analysis_neighborhoods,hsoc_zones_as_of_2018-06-05,owed_public_spaces,central_market/tenderloin_boundary_polygon_-_updated,parks_alliance_cpsi_(27+tl_sites),esncag_-_boundary_file,"areas_of_vulnerability,_2016"
0,2019/05/01 01:00:00 AM,2019/05/01,01:00,2019,Wednesday,2019/06/12 08:27:00 PM,81097515200,810975,190424067,191634131.0,II,Initial,NaN,15200,Offences Against The Family And Children,Other,Domestic Violence (secondary only),Open or Active,40TH AVE \ IRVING ST,27789000.0,Taraval,Sunset/Parkside,4.0,37.762569,-122.499627,"(37.76256939715695, -122.49962745519909)",39.0,10.0,7.0,35.0,NaN,NaN,NaN,NaN,NaN,1.0
1,2019/06/22 07:45:00 AM,2019/06/22,07:45,2019,Saturday,2019/06/22 08:05:00 AM,81465564020,814655,190450880,191730737.0,II,Initial,NaN,64020,Non-Criminal,Other,Mental Health Detention,Open or Active,06TH ST \ MINNA ST,24285000.0,Southern,South of Market,6.0,37.780535,-122.408161,"(37.7805353858225, -122.40816079455212)",32.0,1.0,10.0,34.0,1.0,NaN,1.0,NaN,NaN,2.0
2,2019/06/03 04:16:00 PM,2019/06/03,16:16,2019,Monday,2019/06/03 04:16:00 PM,80769875000,807698,190397016,191533509.0,IS,Initial Supplement,NaN,75000,Missing Person,Missing Person,Found Person,Open or Active,EGBERT AVE \ INGALLS ST,20447000.0,Bayview,Bayview Hunters Point,10.0,37.721600,-122.390745,"(37.72159985216247, -122.39074534279013)",88.0,2.0,9.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0
3,2018/11/16 04:34:00 PM,2018/11/16,16:34,2018,Friday,2018/11/16 04:34:00 PM,73857915041,738579,180870806,183202539.0,IS,Initial Supplement,NaN,15041,Offences Against The Family And Children,Family Offenses,Elder Adult or Dependent Abuse (not Embezzleme...,Cite or Arrest Adult,MERCHANT ST \ KEARNY ST,24773000.0,Central,Chinatown,3.0,37.794860,-122.404876,"(37.794859532228344, -122.40487561154785)",104.0,6.0,3.0,6.0,NaN,18.0,NaN,NaN,NaN,2.0
4,2019/05/27 02:25:00 AM,2019/05/27,02:25,2019,Monday,2019/05/27 02:55:00 AM,80509204134,805092,190378555,191470256.0,II,Initial,NaN,4134,Assault,Simple Assault,Battery,Open or Active,LAGUNA ST \ UNION ST,26583000.0,Northern,Marina,2.0,37.797716,-122.430559,"(37.79771621229674, -122.43055896140594)",15.0,4.0,6.0,13.0,NaN,NaN,NaN,NaN,NaN,1.0


In [3]:
# Keep only the relevant columns
keep_cols = ['incident_date', 'incident_time', 'incident_year', 'incident_day_of_week', 'report_datetime',
             'filed_online', 'incident_category', 'incident_subcategory', 'police_district', 'analysis_neighborhood',
             'latitude', 'longitude', 'areas_of_vulnerability,_2016']
df = df[keep_cols].copy()
print(df.shape)

(333016, 13)


In [4]:
# Add month and day
df['incident_month'] = pd.to_datetime(df['incident_date'], format = "%Y/%m/%d").apply(lambda x:x.month)
df['incident_day'] = pd.to_datetime(df['incident_date'], format = "%Y/%m/%d").apply(lambda x:x.day)
df['incident_hour'] = df['incident_time'].apply(lambda x:x[:2])
df.head()

,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,filed_online,incident_category,incident_subcategory,police_district,analysis_neighborhood,latitude,longitude,"areas_of_vulnerability,_2016",incident_month,incident_day,incident_hour
0,2019/05/01,01:00,2019,Wednesday,2019/06/12 08:27:00 PM,NaN,Offences Against The Family And Children,Other,Taraval,Sunset/Parkside,37.762569,-122.499627,1.0,5,1,01
1,2019/06/22,07:45,2019,Saturday,2019/06/22 08:05:00 AM,NaN,Non-Criminal,Other,Southern,South of Market,37.780535,-122.408161,2.0,6,22,07
2,2019/06/03,16:16,2019,Monday,2019/06/03 04:16:00 PM,NaN,Missing Person,Missing Person,Bayview,Bayview Hunters Point,37.721600,-122.390745,2.0,6,3,16
3,2018/11/16,16:34,2018,Friday,2018/11/16 04:34:00 PM,NaN,Offences Against The Family And Children,Family Offenses,Central,Chinatown,37.794860,-122.404876,2.0,11,16,16
4,2019/05/27,02:25,2019,Monday,2019/05/27 02:55:00 AM,NaN,Assault,Simple Assault,Northern,Marina,37.797716,-122.430559,1.0,5,27,02


In [5]:
# Fix nan valeus
print(df.isna().sum())

df['filed_online'] = df['filed_online'].fillna(False)
for col in ['incident_category', 'incident_subcategory', 'analysis_neighborhood', 'areas_of_vulnerability,_2016']:
    df[col] = df[col].fillna('unknown')

for col in ['latitude', 'longitude']:
    df[col] = df[col].fillna(-99999)
print(df.isna().sum())

incident_date                        0
incident_time                        0
incident_year                        0
incident_day_of_week                 0
report_datetime                      0
filed_online                    260960
incident_category                   46
incident_subcategory                46
police_district                      0
analysis_neighborhood            17897
latitude                         17832
longitude                        17832
areas_of_vulnerability,_2016     17961
incident_month                       0
incident_day                         0
incident_hour                        0
dtype: int64
incident_date                   0
incident_time                   0
incident_year                   0
incident_day_of_week            0
report_datetime                 0
filed_online                    0
incident_category               0
incident_subcategory            0
police_district                 0
analysis_neighborhood           0
latitude               

In [40]:
# # Calculate Zipcode and other useful information
# from uszipcode import SearchEngine
# search = SearchEngine(simple_zipcode=True)

# df_len = len(df)
# info = {'zipcode':[], 'population':[], 'population_density':[], 'housing_units':[], 'occupied_housing_units':[],
#         'median_home_value':[], 'median_household_income':[]}
# for i, (la, lo) in enumerate(zip(df['latitude'], df['longitude'])):
#     if i % (df_len//10) == 0: print(f"Finished: {i}/{df_len}")
#     result = search.by_coordinates(la, lo, radius=30, returns=1)
#     if len(result) != 0:
#         result = result[0]
#         for k in info.keys():
#             info[k].append(getattr(result, k))
#     else:
#         for k in info.keys():
#             info[k].append(np.nan)

# info = pd.DataFrame(info)
# info.to_csv('data/raw/zip_other_info.csv', index=None)

Finished: 0/333016
Finished: 33301/333016
Finished: 66602/333016
Finished: 99903/333016
Finished: 133204/333016
Finished: 166505/333016
Finished: 199806/333016
Finished: 233107/333016
Finished: 266408/333016
Finished: 299709/333016
Finished: 333010/333016


In [6]:
# Append and save to disk
info = pd.read_csv('data/raw/zip_other_info.csv')
df = df.join(info)
print(df.shape)
df.to_csv('data/mst/clean_police_reports.csv', index = None)

(333016, 23)
